In [1]:
from libs import *
from os.path import join
import torch
from tqdm import tqdm
dataSetPath = "dataSet/"

下载数据

In [2]:
token = "b54bfb5fc70a78e4962b8c55911b93a0a4ddd4c764115aeee3c301a3"
downLoadData = DownloadData(Token=token, dataSetPath=dataSetPath)
downLoadData.storeData()

100%|██████████| 3/3 [00:04<00:00,  1.46s/it]


处理数据

In [3]:
dataProcess = DataProcess(dataSetPath=dataSetPath)
dataProcess.loadData()
dataProcess.dropDataFrame(labels=["ts_code", "trade_date"], dim="Parameter")
dataProcess.copyDataFrame(label="close", dim="Parameter")
dataProcess.minMaxNormal()
dataProcess.storeData()

Load daily data: 100%|██████████| 3159/3159 [00:10<00:00, 289.36it/s]


KeyboardInterrupt: 

加载数据

In [2]:
dataSet = DataSet(dataSetPath=join(dataSetPath, "data/marketData/data.nc"))
dataSet.setLength(trainDays=360, targetDays=30)  # 有默认时间长度，可以不用该函数设置(这里仅作演示)
dataLoader = DataLoader(dataSet=dataSet)

模型训练

In [3]:
module = modules.ResNetLSTM()
module.cuda()
output = 0

In [4]:
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
torch.cuda.empty_cache()
torch.set_grad_enabled(False)
for train_data, target_data in tqdm(dataLoader):
    train_data = train_data.cuda()
    train_data = train_data.reshape(360, 1, 120, 120)
    output = module(train_data)

    torch.cuda.empty_cache()
    break

  0%|          | 0/2770 [00:02<?, ?it/s]


In [ ]:
print(output.shape)

获取预测数据

In [ ]:
import torch


数据可视化

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

数据分析